In [1]:
import geopandas as gpd
import pandas as pd
import warnings
from tqdm.contrib.concurrent import process_map
warnings.filterwarnings('ignore')
from multiprocessing import Process, Manager

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
from simpledbf import Dbf5
dbf = Dbf5('data/input/Master_Data/mfd_2020_1.dbf')
df_master = dbf.to_dataframe()
df_master['idkab'] = df_master['PROP'] + df_master['KAB']

PyTables is not installed. No support for HDF output.


In [3]:
def generate_data_kabupaten(kdkab):
    print('[+] Proses membuat data kabupaten!')
    import os
    if not os.path.exists('data/input/Master_Data/DESA_20201_BYINDO_'+kdkab+'.gpkg'):
        data_desa=gpd.read_file('data/input/Master_Data/DESA_20201_BYINDO.gpkg').to_crs('ESRI:54034')
        d = data_desa[(data_desa['kdprov'] == kdkab[:2]) & (data_desa['kdkab'] == kdkab[2:4])]
        d.to_file('data/input/Master_Data/DESA_20201_BYINDO_'+kdkab+'.gpkg',driver='GPKG')

    if not os.path.exists('data/input/RBI/kdkab_'+kdkab+'.gpkg'):
        data_rai = gpd.read_file('data/input/RBI/kdprov_'+kdkab[:2]+'.gpkg').to_crs('ESRI:54034')
        dr = gpd.sjoin(data_rai,d)
        dr = dr.dissolve()
        d.to_file('data/input/RBI/kdkab_'+kdkab+'.gpkg',driver='GPKG')
    
    if not os.path.exists('data/output/2_grid_degurba/degurba_Sumatera_'+kdkab+'.gpkg'):
        nmfiledegurba = "degurba_Sumatera_"+kdkab[:2]+".gpkg"
        gf = gpd.read_file("data/output/2_grid_degurba/"+nmfiledegurba).to_crs('ESRI:54034')
        gf['RAI'] = 0
        g = gpd.sjoin(gf,d)[['Name', 'pop', 'p_builtup', 'is_U_CENTRE_w_BU', 'is_U_CLUSTER',
               'classification', 'geometry', 'RAI']]
        g.to_file('data/output/2_grid_degurba/degurba_Sumatera_'+kdkab+'.gpkg',driver='GPKG')
    print('[+] Selesai membuat data kabupaten!')

In [4]:
def data_masking_degurba(i):
    global degurba_desa,data_highway,gf,affected_RAI
    degurba_desa=gpd.sjoin(gf,data_desa.query('iddesa==@i'))
    data_highway_j=data_rai.overlay(degurba_desa,how='intersection')[['geometry']]
    data_highway_j['reg']=1
    data_highway_j=data_highway_j.dissolve(by='reg')
    data_highway_j.geometry=data_highway_j.buffer(2000)
    nm_rai=gf.sjoin(data_highway_j)['Name'].tolist()
    affected_RAI.extend(nm_rai)

In [5]:
def main(idkab):
    print('[+] Melakukan perhitungan proses RBI!')
    global gf,data_desa,data_rai,affected_RAI
    gf = gpd.read_file('data/output/2_grid_degurba/degurba_Sumatera_'+idkab+'.gpkg')
    data_desa = gpd.read_file('data/input/Master_Data/DESA_20201_BYINDO_'+idkab+'.gpkg').to_crs('ESRI:54034')
    data_rai = gpd.read_file('data/input/RBI/kdkab_'+idkab+'.gpkg')
    manager=Manager()
    affected_RAI=manager.list()
    process_map(data_masking_degurba,data_desa.iddesa.unique())
    gf[gf.Name.isin(list(affected_RAI))].to_file('data/output/4_RAI/rai_rbi_degurba_'+idkab+'.gpkg',driver='GPKG')

In [6]:
list_kdprov = ['16','18']
gf = None
data_desa = None
affected_RAI = None
data_rai = None

In [7]:
for kdprov in list_kdprov:
    list_kabupaten = df_master[df_master['idkab'].str.startswith(kdprov)]
    for i in list_kabupaten['idkab'].unique():
        import os.path
        if not os.path.exists('data/output/4_RAI/rai_rbi_degurba_'+i+'.gpkg'):
            print('[+] Melakukan proses untuk kabupaten {}'.format(i))
            generate_data_kabupaten(i)
            main(i)
        else:
            print('[+] Kabupaten {} sudah pernah diproses!'.format(i))

[+] Kabupaten 1601 sudah pernah diproses!
[+] Kabupaten 1602 sudah pernah diproses!
[+] Kabupaten 1603 sudah pernah diproses!
[+] Kabupaten 1604 sudah pernah diproses!
[+] Kabupaten 1605 sudah pernah diproses!
[+] Kabupaten 1606 sudah pernah diproses!
[+] Kabupaten 1607 sudah pernah diproses!
[+] Kabupaten 1608 sudah pernah diproses!
[+] Kabupaten 1609 sudah pernah diproses!
[+] Kabupaten 1610 sudah pernah diproses!
[+] Kabupaten 1611 sudah pernah diproses!
[+] Kabupaten 1612 sudah pernah diproses!
[+] Kabupaten 1613 sudah pernah diproses!
[+] Kabupaten 1671 sudah pernah diproses!
[+] Kabupaten 1672 sudah pernah diproses!
[+] Kabupaten 1673 sudah pernah diproses!
[+] Kabupaten 1674 sudah pernah diproses!
[+] Kabupaten 1801 sudah pernah diproses!
[+] Kabupaten 1802 sudah pernah diproses!
[+] Kabupaten 1803 sudah pernah diproses!
[+] Kabupaten 1804 sudah pernah diproses!
[+] Melakukan proses untuk kabupaten 1805
[+] Proses membuat data kabupaten!
[+] Selesai membuat data kabupaten!
[+] M

  0%|          | 0/316 [00:00<?, ?it/s]

[+] Melakukan proses untuk kabupaten 1806
[+] Proses membuat data kabupaten!
[+] Selesai membuat data kabupaten!
[+] Melakukan perhitungan proses RBI!


  0%|          | 0/250 [00:00<?, ?it/s]

[+] Melakukan proses untuk kabupaten 1807
[+] Proses membuat data kabupaten!
[+] Selesai membuat data kabupaten!
[+] Melakukan perhitungan proses RBI!


...

  0%|          | 0/121 [00:00<?, ?it/s]

[+] Melakukan proses untuk kabupaten 1871
[+] Proses membuat data kabupaten!
[+] Selesai membuat data kabupaten!
[+] Melakukan perhitungan proses RBI!


  0%|          | 0/126 [00:00<?, ?it/s]

[+] Melakukan proses untuk kabupaten 1872
[+] Proses membuat data kabupaten!
[+] Selesai membuat data kabupaten!
[+] Melakukan perhitungan proses RBI!


  0%|          | 0/22 [00:00<?, ?it/s]

In [8]:
for kdprov in list_kdprov:
    print('[+] Melakukan proses penggabungan RAI RBI untuk provinsi {}'.format(kdprov))
    
    data_rai = gpd.read_file('data/input/RBI/kdprov_'+kdprov+'.gpkg').to_crs('ESRI:54034').dissolve()
    data_desa = gpd.read_file('data/input/Master_Data/DESA_20201_BYINDO.gpkg').to_crs('ESRI:54034').query('kdprov==@kdprov')

    nmfiledegurba = "degurba_Sumatera_00.gpkg"
    gf = gpd.read_file("data/output/2_grid_degurba/"+nmfiledegurba).to_crs('ESRI:54034')
    gf['RAI'] = 0
    gf = gf.sjoin(data_desa[['geometry']])[gf.columns]
    
    from glob import glob
    degurba_rai = glob('data/output/4_RAI/rai_rbi_degurba_'+kdprov+'*.gpkg')
    data_ = gpd.GeoDataFrame()
    l_name = []
    for i in degurba_rai:
        temp = gpd.read_file(i)['Name']
        l_name.extend(list(temp))
    l_name = list(set(l_name))
    gf['accessed'] = gf.Name.apply(lambda y: 1 if y in l_name else 0)
    gf.to_file('data/output/4_RAI/fin_rai_rbi_degurba_'+kdprov+'.gpkg')
    
    print('[+] Selesai melakukan proses untuk provinsi {}'.format(kdprov))

[+] Melakukan proses penggabungan RAI RBI untuk provinsi 16
[+] Selesai melakukan proses untuk provinsi 16
[+] Melakukan proses penggabungan RAI RBI untuk provinsi 18
[+] Selesai melakukan proses untuk provinsi 18
